# Clustering Models

Clustering is a foundational data analysis task, where members of the data set are sorted into groups or "clusters" according to measured similarities between the objects. According to some quantitative criteria, members of the same cluster are similar and members of distinct clusters are different.

In [ ]:
import numpy as np
import pandas as pd
import graspy
import matplotlib.pyplot as plt
%matplotlib inline

## Automatic Gaussian Mixture Model (AUTOGMM)

The Automatic Gaussian Mixture Model or AutoGMM is a clustering algorithm that uses Sklearn's hierarchical agglomerative clustering and then Gaussian mixtured model (GMM) fitting. Different combinations of agglomeration, GMM,and cluster numbers are used in the algorithm, and the clustering with the best selection criterion (bic/aic) is chosen.

Clustering is a foundational data analysis task, where members of the data set are sorted into groups or "clusters" according to measured similarities between the objects. According to some quantitative criteria, members of the same cluster are similar and members of distinct clusters are different. 

This algorithm is a Gaussian mixture model (GMM), a statistical model of clustered data that, simply put, is a composition of multiple normal distributions. Each cluster has a weight $w_k$ assigned to it, and the combined probability distribution, $f(x)$, is of the form:

$f(x) = \sum\limits_{k = 1}^K {w_{k}f_{k}(x)} = \sum\limits_{k = 1}^K {\frac{w_{k}}{(2\pi)^{\frac{d}{2}}|\sum_{k}|^{-\frac{1}{2}}}e^{[\frac{1}{2}(x - \mu_{k})^{T}\sum_{k}^{-1}(x - \mu_{k})]}}$

where $k$ is the total number of clusters and $d$ is the dimensionality of the data.

Expectation Maximization (EM) algorithms are then run to estimate model parameters and the fitted GMM is used to cluster the data.

Let's start with a simple example,  where the algorithm uses all possible forms of clustering on a basic set of ten samples.

In [ ]:
from graspy.cluster.autogmm import AutoGMMCluster

# Ex
x = np.identity(10)
AutoGMM = AutoGMMCluster(min_components=3, affinity="all")
AutoGMM.fit(x)

The results and all calculations are presented as a dataframe.

In [ ]:
AutoGMM.results_

Now, let's look at a more complex example with carefully created synthetic data.

In [ ]:
import csv
import time
import random

from sklearn.datasets import load_iris
from sklearn import datasets
import matplotlib.colors as colors

from graspy.cluster.autogmm import AutoGMMCluster
from graspy.simulations import sbm
from graspy.plot import heatmap

np.random.seed(10)


# Using all forms of clustering
ks = [i for i in range(1,21)]
affinities = 'all'
linkages = 'all'
covariance_types='all'
    
# Creating Synthetic Data
x = np.genfromtxt('/home/caseypw/data/synthetic.csv', delimiter=',',skip_header=0)
x = x[:,np.arange(1,x.shape[1])]
c_true = np.genfromtxt('/home/caseypw/data/synthetic.csv', delimiter=',', usecols = (0),skip_header=0)

def make_bic_plots(results,best_cov,best_k_bic,best_bic):
    #plot of all BICS*******************************
    titles = ['Full','Tied','Diagonal','Spherical']
    bics = np.zeros((44,20))
    cov_types = ['full','tied','diag','spherical']
    for i,cov_type in enumerate(cov_types):
        bics[i*11+0,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'none')]['bic/aic'].values
        bics[i*11+1,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'euclidean') & (results['linkage'] == 'ward')]['bic/aic'].values
        bics[i*11+2,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'euclidean') & (results['linkage'] == 'complete')]['bic/aic'].values
        bics[i*11+3,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'euclidean') & (results['linkage'] == 'average')]['bic/aic'].values
        bics[i*11+4,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'euclidean') & (results['linkage'] == 'single')]['bic/aic'].values
        bics[i*11+5,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'manhattan') & (results['linkage'] == 'complete')]['bic/aic'].values
        bics[i*11+6,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'manhattan') & (results['linkage'] == 'average')]['bic/aic'].values
        bics[i*11+7,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'manhattan') & (results['linkage'] == 'single')]['bic/aic'].values
        bics[i*11+8,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'cosine') & (results['linkage'] == 'complete')]['bic/aic'].values
        bics[i*11+9,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'cosine') & (results['linkage'] == 'average')]['bic/aic'].values
        bics[i*11+10,:] = -results.loc[(results['covariance_type'] == cov_type) &
            (results['affinity'] == 'cosine') & (results['linkage'] == 'single')]['bic/aic'].values


    labels = {0:'none',1:'l2/ward',2:'l2/complete',3:'l2/average',4:'l2/single',
            5:'l1/complete',6:'l1/average',7:'l1/single',8:'cos/complete',
            9:'cos/average',10:'cos/single'}
        
    fig, ((ax0,ax1),(ax2,ax3)) = plt.subplots(2,2,sharey='row',sharex='col',figsize=(7,6))
    for row in np.arange(bics.shape[0]):
        if all(bics[row,:]==-np.inf):
            continue
        if row<=10:
            ax0.plot(np.arange(1,len(ks)+1),bics[row,:])
        elif row<=21:
            ax1.plot(np.arange(1,len(ks)+1),bics[row,:],label=labels[row%11])
        elif row<=32:
            ax2.plot(np.arange(1,len(ks)+1),bics[row,:])
        elif row<=43:
            ax3.plot(np.arange(1,len(ks)+1),bics[row,:])
    
    #plot line indicating chosen model
    if best_cov == 'full':
        ylims = ax0.get_ylim()
        ax0.plot([best_k_bic,best_k_bic],[ylims[0],best_bic],color='black',linestyle='dashed',linewidth=2)
    elif best_cov == 'tied':
        ylims = ax1.get_ylim()
        ax1.plot([best_k_bic,best_k_bic],[ylims[0],best_bic],color='black',linestyle='dashed',linewidth=2)
    elif best_cov == 'diag':
        ylims = ax2.get_ylim()
        ax2.plot([best_k_bic,best_k_bic],[ylims[0],best_bic],color='black',linestyle='dashed',linewidth=2)
    elif best_cov == 'spherical':
        ylims = ax3.get_ylim()
        ax3.plot([best_k_bic,best_k_bic],[ylims[0],best_bic],color='black',linestyle='dashed',linewidth=2)
    
    fig.text(0.5, 0.04, 'Number of Components', ha='center',fontsize=18,fontweight='bold')
    fig.text(0.01, 0.5, 'BIC', va='center', rotation='vertical',fontsize=18,fontweight='bold')
    
    ax0.set_title(titles[0],fontsize=22,fontweight='bold')
    ax0.locator_params(axis='y',tight=True,nbins=4)
    ax0.set_yticklabels(ax0.get_yticks(),fontsize=18)

    ax1.set_title(titles[1],fontsize=22,fontweight='bold')
    legend = ax1.legend(loc='best',bbox_to_anchor=(1.25, 1.25, 0, 0),title='Agglomeration\nMethod',fontsize=12)
    plt.setp(legend.get_title(),fontsize=14)

    ax2.set_title(titles[2],fontsize=22,fontweight='bold')
    ax2.set_xticks(range(0,21,4))
    ax2.set_xticklabels(ax2.get_xticks(),fontsize=18)
    ax2.locator_params(axis='y',tight=True,nbins=4)
    ax2.set_yticklabels(ax2.get_yticks(),fontsize=18)
    

    ax3.set_title(titles[3],fontsize=22,fontweight='bold')
    ax3.set_xticks(range(0,21,4))
    ax3.set_xticklabels(ax3.get_xticks(),fontsize=18)

# Run algorithm
pyc = AutoGMMCluster(min_components=ks[0],max_components=ks[len(ks)-1],
    affinity=affinities,linkage=linkages,covariance_type=covariance_types,
    random_state=0)

pyc.fit(x,c_true)

# Calculated covariates
combo = [pyc.affinity_,pyc.linkage_,pyc.covariance_type_]
k = pyc.n_components_
reg = pyc.reg_covar_
bic = -pyc.criter_
results = pyc.results_
ari = pyc.ari_
c_hat_autogmm = pyc.predictions

print('Info:')
print('Best model: ' + str(combo))
print('Best reg: ' + str(reg))
print('Best k: ' + str(k))
print('Best BIC: ' + str(bic))
print('Best ari: ' + str(ari))

# BIC Plots
make_bic_plots(results,combo[2],k,bic)


In [ ]:
print(pyc.results_)

## K-Means Clustering (kclust)

kclust is a clustering algorithm that finds the optimal model by using all algorithms and calculating the lowest silhouette score from Sklearn.

Here is the same simple example.

In [ ]:
from graspy.cluster.kclust import KMeansCluster

# Ex
x = np.identity(10)
KMeansClust = KMeansCluster(max_clusters=5)
KMeansClust.fit(x)

KMeansClust.model_

Here is the same complex example.

In [ ]:
# Creating Synthetic Data
x = np.genfromtxt('/home/caseypw/data/synthetic.csv', delimiter=',',skip_header=0)
x = x[:,np.arange(1,x.shape[1])]
c_true = np.genfromtxt('/home/caseypw/data/synthetic.csv', delimiter=',', usecols = (0),skip_header=0)

KMeansClust.fit(x, c_true)

KMeansClust.model_

## GraspyClust (gclust)

gclust is the last clustering algorithm and it is purely a GMM approach, with no agglomerative clustering.

Last simple example.

In [ ]:
from graspy.cluster.gclust import GaussianCluster

# Ex
x = np.identity(10)
GClust = GaussianCluster()
GClust.fit(x)

GClust.model_

Complex example.

In [ ]:
# Creating Synthetic Data
x = np.genfromtxt('/home/caseypw/data/synthetic.csv', delimiter=',',skip_header=0)
x = x[:,np.arange(1,x.shape[1])]
c_true = np.genfromtxt('/home/caseypw/data/synthetic.csv', delimiter=',', usecols = (0),skip_header=0)

GClust.fit(x, c_true)

GClust.model_